In [1]:
%run basics.ipynb

Populating the interactive namespace from numpy and matplotlib


Loading BokehJS ...

Using Theano backend.


In [2]:
df = pd.read_pickle('udacity_courses.df')
print(df.columns)
df.head(2)

Index([u'id', u'title', u'homepage', u'subtitle', u'level', u'starter',
       u'image', u'banner_image', u'teaser_video', u'summary',
       u'short_summary', u'required_knowledge', u'expected_learning',
       u'featured', u'syllabus', u'faq', u'full_course_available',
       u'expected_duration', u'expected_duration_unit', u'new_release',
       u'transcripts_url', u'transcripts_zip_fn', u'transcript'],
      dtype='object')


,id,title,homepage,subtitle,level,starter,image,banner_image,teaser_video,summary,...,featured,syllabus,faq,full_course_available,expected_duration,expected_duration_unit,new_release,transcripts_url,transcripts_zip_fn,transcript
0,bio110,Tales from the Genome,https://www.udacity.com/course/tales-from-the-...,Introduction to Genetics for Beginners,beginner,False,https://lh5.ggpht.com/Fh07_XbT61CGDWsQoOMQaIeH...,https://lh4.ggpht.com/_AHqr0EXcLBzF0rrwtBagE0t...,https://www.youtube.com/watch?v=sEXrjh-tme8,This course is a journey into the biology of t...,...,False,## Lesson 1: Introduction to traits and heredi...,NaN,False,3,months,False,https://www.udacity.com/course/progress#!/c-bi...,transcripts_zips/bio110.zip,In the same way that our genomes influence tr...
1,cs046,Intro to Java Programming,https://www.udacity.com/course/intro-to-java-p...,Building Programs with Classes & Objects,beginner,False,https://lh4.ggpht.com/9ytiUdz0QYHwuMJFTXcNXZn4...,https://lh3.ggpht.com/AdAALPYhCsWuIvDl0ZY6zIW2...,https://www.youtube.com/watch?v=Wsp5Rrenoq4,"In this introductory course, you'll learn and ...",...,False,"### Lesson 1: Introduction to Computers, Progr...",NaN,True,4,months,False,https://www.udacity.com/course/progress#!/c-cs046,transcripts_zips/cs046.zip,"Hi, I'm Cay. I'm Sara. We'll be the instructo..."


In [3]:
df_prereq = pd.read_csv('udacity_prereq_graph.csv')
df_prereq.head(2)

,u_id,u_pre_req_id
0,cs222,cs101
1,cs222,ph100


In [4]:
def is_prereq(a, b, preq):
    a_p = preq[preq['u_id'] == a]
    if len(a_p) > 0:
        if b in a_p['u_pre_req_id'].values.tolist():
            return True
        else:
            return False
    else:
        return False

In [5]:
fields = ['title', 'subtitle', 'summary', 'syllabus', 'transcript']
print(df[fields].count())
vectorizer = TfidfVectorizer(stop_words='english', max_df=0.5, min_df=5)
all_text = df[fields[0]].copy()
for c in fields[1:]:
    all_text += ' ' + df[c]
all_text = all_text.apply(text_cleanup)
print(all_text.head(2))
vectorizer.fit(all_text)
filter_w = set(vectorizer.get_feature_names())
print('# distinct words:', len(filter_w))
print('# excluded words:', len(vectorizer.stop_words_))

title         129
subtitle      127
summary       129
syllabus      129
transcript    129
dtype: int64
0    tales from the genome introduction to genetics...
1    intro to java programming building programs wi...
Name: title, dtype: object
('# distinct words:', 9813)
('# excluded words:', 30619)


In [6]:
digiz = Digitizer()
max_words = 10000
text_series = list()
for i in fields:
    ser = df[i].copy()
    ser = ser.apply(text_cleanup, args=(filter_w, ))
    print(i, ser.apply(lambda x: len(x.split())).max())
    print(ser.head(2))
    dig_ser = digiz.series_digitizer(ser, max_len=max_words)
    text_series.append(dig_ser)
    print(dig_ser.head(2))

('title', 4)
0        genome
1    intro java
Name: title, dtype: object
0    [1, 0, 0, 0]
1    [2, 3, 0, 0]
Name: title, dtype: object
('subtitle', 4)
0    genetics beginners
1              programs
Name: subtitle, dtype: object
0    [121, 96, 0, 0]
1       [4, 0, 0, 0]
Name: subtitle, dtype: object
('summary', 69)
0    journey biology genome highlight scientific so...
1    introductory essential java oriented programme...
Name: summary, dtype: object
0    [234, 235, 1, 236, 237, 123, 238, 239, 240, 24...
1    [242, 243, 3, 31, 244, 4, 3, 245, 246, 0, 0, 0...
Name: summary, dtype: object
('syllabus', 244)
0    traits trait variation categorization inherita...
1    computers algorithms java graphics fundamental...
Name: syllabus, dtype: object
0    [241, 1219, 1220, 1221, 1222, 1223, 171, 1224,...
1    [524, 5, 3, 13, 666, 1271, 1272, 5, 1088, 1222...
Name: syllabus, dtype: object
('transcript', 40593)
0    influence traits eye hair cancer influence dru...
1    sara instructors processo

In [7]:
text_series[0].head(2)

0    [1, 0, 0, 0]
1    [2, 3, 0, 0]
Name: title, dtype: object

In [8]:
all_text = text_series[0].copy()
text_block_len = dict()
text_block_len[fields[0]] = len(text_series[0].iloc[0])
for i, f in zip(text_series[1:], fields[1:]):
    all_text += i
    text_block_len[f] = len(i.iloc[0])
df['digitized_text'] = all_text
max_len = df['digitized_text'].apply(len).max()
print(max_len)
df['digitized_text'] = df['digitized_text'].apply(lambda x: np.array(x))
df['digitized_text'].head(10)
for key, val in text_block_len.iteritems():
    print(key, 'len:', val)

10321
('syllabus', 'len:', 244)
('summary', 'len:', 69)
('subtitle', 'len:', 4)
('transcript', 'len:', 10000)
('title', 'len:', 4)


In [9]:
n_words = digiz.num_words()
print('#words:', n_words)
print('seq len:', max_len)
df['digitized_text'].head(2)
assert all(df['digitized_text'].apply(lambda x: len(x) == max_len).tolist())

('#words:', 9809)
('seq len:', 10321)


In [10]:
def get_nn(n_words, input_len):   
    dim = 10
    emb = Embedding(input_dim=n_words, output_dim=dim, mask_zero=True)
    c = Input(shape=(max_len,), dtype='int32', name='course')
    
    dropout = Dropout(0.8)(emb(c))
    
    c_act = Activation('relu')(dropout)
    
    c_lstm = GRU(10, W_regularizer=WeightRegularizer(l1=0., l2=0.5))(c_act)
    
    dense_100 = Dense(10, activation='relu', 
                      W_regularizer=WeightRegularizer(l1=0., l2=0.5))(c_lstm)
    dense_1 = Dense(1, activation='relu')(dense_100)
    
    model = Model(c, dense_1)
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

In [11]:
nn = get_nn(n_words, max_len)
print(nn.summary())
#embedded_text = nn.predict(df['digitized_text'].values.tolist())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
course (InputLayer)              (None, 10321)         0                                            
____________________________________________________________________________________________________
embedding_1 (Embedding)          (None, 10321, 10)     98090       course[0][0]                     
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 10321, 10)     0           embedding_1[0][0]                
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 10321, 10)     0           dropout_1[0][0]                  
___________________________________________________________________________________________

In [12]:
y = df['level'].apply(lambda x: 0 if x=='beginner' else (1 if x == 'intermediate' else 2))
print(y.value_counts())
df['level'].value_counts()

1    66
2    33
0    30
Name: level, dtype: int64


intermediate    66
advanced        32
beginner        30
Name: level, dtype: int64

In [13]:
train_res = list()
test_res = list()

n_shuffles = 1

for train_idx, test_idx in StratifiedShuffleSplit(y, n_iter=10, test_size=0.2, random_state=2016):
    X_train = df.iloc[train_idx]['digitized_text'].values.tolist()
    y_train = y.iloc[train_idx].values.tolist()
    print(len(X_train))
    for s in range(n_shuffles):
        for i in range(len(train_idx)):
            orig_x = X_train[i]
            orig_y = y_train[i]
            f_start = 0
            f_end = 0
            shuffled = list()
            for f in fields:
                f_start = f_end
                f_end += text_block_len[f]
                f = orig_x[f_start:f_end].tolist()
                random.shuffle(f)
                shuffled.extend(f)
            assert len(shuffled) == len(orig_x)
            X_train.append(np.array(shuffled))
            y_train.append(orig_y)
    
    X_train = np.vstack(X_train)
    y_train = np.array(y_train)
    print(len(X_train))
    
    X_valid = np.vstack(df.iloc[test_idx]['digitized_text'].values)
    y_valid = y.iloc[test_idx].values
    print(y.iloc[test_idx].value_counts())
    nn = get_nn(n_words, max_len)
    class_dist = y.iloc[train_idx].value_counts().to_dict()
    class_w = {i:(len(y_train)/(len(class_dist) + n_shuffles * len(class_dist)))/j for i,j in class_dist.iteritems()}
    hist = nn.fit(X_train, y_train, class_weight=class_w,
                  batch_size=8,
                  validation_data=(X_valid, y_valid),
                  shuffle=True,
                  nb_epoch=20, 
                  verbose=1)
    plot_nn_hist(hist)
    y_train_pred = nn.predict(X_train)
    y_valid_pred = nn.predict(X_valid)
    train_res.extend(evaluate(y_train, y_train_pred))
    test_res.extend(evaluate(y_valid, y_valid_pred))
    viz_res(train_res, 'training')
    viz_res(test_res, 'validation')

103
206
1    13
2     7
0     6
Name: level, dtype: int64
Train on 206 samples, validate on 26 samples
Epoch 1/20
206/206 [==============================] - 53s - loss: 1.7762 - mean_absolute_error: 0.9515 - val_loss: 1.0862 - val_mean_absolute_error: 0.8885
Epoch 2/20
184/206 [=========================>....] - ETA: 5s - loss: nan - mean_absolute_error: nan

KeyboardInterrupt: 

In [ ]:
viz_res(train_res, 'training')
viz_res(test_res, 'validation')